In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install tensorflow
!pip install keras

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_gen=ImageDataGenerator(rescale=1./255)

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vYcokdlj6X5OxEo1Dk0WHI1FBZqV6Gel71LBXNJs0uxS',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'fertilizersrecommendationsystemfo-donotdelete-pr-fzjg5do8pezxnv'
object_key = 'Fertilizers_Recommendation_ System_For_Disease_ Prediction.zip'

streaming_body_3 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_3.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
pwd

'/home/wsuser/work'

In [8]:
import os
os.listdir('/home/wsuser/work')

['Dataset Plant Disease', '.virtual_documents']

In [9]:
x_train=train_gen.flow_from_directory('/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/train',target_size=(128,128),class_mode='categorical',batch_size=32)
x_test=test_gen.flow_from_directory('/home/wsuser/work/Dataset Plant Disease/fruit-dataset/fruit-dataset/test',target_size=(128,128),class_mode='categorical',batch_size=32)

Found 5384 images belonging to 6 classes.
Found 1686 images belonging to 6 classes.


In [10]:
x_train.class_indices

{'Apple___Black_rot': 0,
 'Apple___healthy': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3,
 'Peach___Bacterial_spot': 4,
 'Peach___healthy': 5}

In [11]:
model=Sequential()

In [12]:
model.add(Convolution2D(32,(3,3),activation="relu",input_shape=(128,128,3)))

In [13]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [14]:
model.add(Flatten())

In [15]:
model.add(Dense(500,activation="relu"))
model.add(Dense(500,activation="relu"))
model.add(Dense(300,activation="relu"))
model.add(Dense(300,activation="relu"))

In [16]:
model.add(Dense(6,activation="softmax"))

In [17]:
model.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")

In [18]:
model.fit(x_train,epochs=5,validation_data=x_test,steps_per_epoch=len(x_train),validation_steps=len(x_test))

Epoch 1/5
169/169 [==============================] - 141s 831ms/step - loss: 1.0577 - accuracy: 0.7056 - val_loss: 0.3248 - val_accuracy: 0.8861
Epoch 2/5
169/169 [==============================] - 139s 820ms/step - loss: 0.2871 - accuracy: 0.9095 - val_loss: 0.3249 - val_accuracy: 0.8837
Epoch 3/5
169/169 [==============================] - 140s 827ms/step - loss: 0.2328 - accuracy: 0.9251 - val_loss: 0.1594 - val_accuracy: 0.9348
Epoch 4/5
169/169 [==============================] - 141s 833ms/step - loss: 0.1988 - accuracy: 0.9298 - val_loss: 0.1541 - val_accuracy: 0.9484
Epoch 5/5
169/169 [==============================] - 141s 837ms/step - loss: 0.1691 - accuracy: 0.9402 - val_loss: 0.1246 - val_accuracy: 0.9609


In [19]:
model.save("Fruit.h5")

In [20]:
!zip -r fruit_disease_prediction_model.zip Fruit.h5

  adding: Fruit.h5 (deflated 22%)


In [21]:
ls

'Dataset Plant Disease'/   fruit_disease_prediction_model.zip   Fruit.h5


In [22]:
!pip install watson-machine-learning-client --upgrade

In [23]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"E9ry9yQOMf_w6jWWNUZsHS3ESFmb0yHH0_1x2OWK735v"
}

client=APIClient(wml_credentials)

In [24]:
client=APIClient(wml_credentials)

In [25]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==space_name)['metadata']['id'])

In [26]:
space_uid=guid_from_space_name(client,'Fertilizers_Recommendation_System_For_Disease_Prediction')
print('Space UID = '+space_uid)

Space UID = 05529a03-f43c-4226-859a-24da5595a26a


In [27]:
client.set.default_space(space_uid)

'SUCCESS'

In [28]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [29]:
software_spec_uid=client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [30]:
model_details=client.repository.store_model(model='fruit_disease_prediction_model.zip',meta_props={
    client.repository.ModelMetaNames.NAME: "CNN-Fruit",
    client.repository.ModelMetaNames.TYPE: "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,})

model_id=client.repository.get_model_id(model_details)

In [31]:
model_id

'981efcad-1b3a-4acd-8b51-6e359b776334'

In [32]:
client.repository.download(model_id,"new_model.zip")

Successfully saved model content to file: 'new_model.zip'


'/home/wsuser/work/new_model.zip'